In [1]:
import pandas as pd
import sqlite3

In [2]:
pd.options.display.max_columns = None

In [3]:
conn = sqlite3.connect('sakila.db')

In [4]:
cursor = conn.cursor()
cursor.execute('''
SELECT * FROM sqlite_master WHERE type = 'table'
''')
data = cursor.fetchall()
columns = [col[0] for col in cursor.description]
cursor.close()

In [5]:
data_df = pd.DataFrame(data, columns=columns)
data_df

,type,name,tbl_name,rootpage,sql
0,table,actor,actor,2,"CREATE TABLE ""actor"" (\n""index"" INTEGER,\n ""a..."
1,table,store,store,7,"CREATE TABLE ""store"" (\n""index"" INTEGER,\n ""s..."
2,table,address,address,9,"CREATE TABLE ""address"" (\n""index"" INTEGER,\n ..."
3,table,category,category,26,"CREATE TABLE ""category"" (\n""index"" INTEGER,\n ..."
4,table,city,city,28,"CREATE TABLE ""city"" (\n""index"" INTEGER,\n ""ci..."
5,table,country,country,39,"CREATE TABLE ""country"" (\n""index"" INTEGER,\n ..."
6,table,customer,customer,43,"CREATE TABLE ""customer"" (\n""index"" INTEGER,\n ..."
7,table,film_actor,film_actor,63,"CREATE TABLE ""film_actor"" (\n""index"" INTEGER,\..."
8,table,film_category,film_category,126,"CREATE TABLE ""film_category"" (\n""index"" INTEGE..."
9,table,inventory,inventory,140,"CREATE TABLE ""inventory"" (\n""index"" INTEGER,\n..."


In [6]:
pd.read_sql_query(sql = '''
SELECT * FROM actor;
''', con = conn)

,index,actor_id,first_name,last_name,last_update
0,0,1,Penelope,Guiness,2013-05-26 14:47:57.620000
1,1,2,Nick,Wahlberg,2013-05-26 14:47:57.620000
2,2,3,Ed,Chase,2013-05-26 14:47:57.620000
3,3,4,Jennifer,Davis,2013-05-26 14:47:57.620000
4,4,5,Johnny,Lollobrigida,2013-05-26 14:47:57.620000
...,...,...,...,...,...
195,195,196,Bela,Walken,2013-05-26 14:47:57.620000
196,196,197,Reese,West,2013-05-26 14:47:57.620000
197,197,198,Mary,Keitel,2013-05-26 14:47:57.620000
198,198,199,Julia,Fawcett,2013-05-26 14:47:57.620000


Question 1:

Are there any films for rent with no actors?

In [10]:
pd.read_sql_query(sql = '''
SELECT f.film_id AS film_id, f.title AS title, c.name AS category, COUNT(i.inventory_id) AS inventory_count
FROM film AS f
LEFt JOIN film_category AS fc
ON f.film_id = fc.film_id
LEFt JOIN category AS c
ON fc.category_id = c.category_id
LEFT JOIN film_actor AS fa
ON f.film_id = fa.film_id 
LEFT JOIN actor AS a
ON fa.actor_id = a.actor_id
JOIN inventory AS i
ON i.film_id = f.film_id
WHERE a.actor_id IS NULL
GROUP BY 1, 2, 3
ORDER BY 4 DESC
;
''', con = conn)

,film_id,title,category,count
0,803,Slacker Liaisons,Drama,6
1,323,Flight Lies,Sports,4
2,257,Drumline Cyclone,Travel,3


Solution:

SELECT f.film_id AS film_id, f.title AS title, c.name AS category, COUNT(i.inventory_id) AS inventory_count
FROM film AS f
LEFt JOIN film_category AS fc
ON f.film_id = fc.film_id
LEFt JOIN category AS c
ON fc.category_id = c.category_id
LEFT JOIN film_actor AS fa
ON f.film_id = fa.film_id 
LEFT JOIN actor AS a
ON fa.actor_id = a.actor_id
JOIN inventory AS i
ON i.film_id = f.film_id
WHERE a.actor_id IS NULL
GROUP BY 1, 2, 3
ORDER BY 4 DESC;

In [24]:
pd.read_sql_query(sql = '''
SELECT *
FROM film_actor
WHERE film_id IN (803,323,257)
;
''', con = conn)

,index,actor_id,film_id,last_update
